# Text Generation Project


## Abstract

<b> Write an abstract here, dude. </b>

## Introduction

<b> Write an intro here, dude. </b>

### Imports

In [3]:
# The usuals
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import sys

# NLTK stuff
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

# Tensorflow stuff
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.python.keras import utils
from tensorflow.keras.callbacks import ModelCheckpoint



### Functions

## Obtain

In [6]:
df = pd.read_csv('Womens Clothing E-Commerce Reviews.csv')
df.drop('Unnamed: 0', axis = 1, inplace = True)
print(df.shape)
df.head()

(23486, 10)


,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
